# torch.nn.ModuleList(modules=None) / .nn.ModuleList()

**ModuleList**:
- PyTorch-ийн дэд модулиудыг жагсаалт хэлбэрээр хадгалах цэгцэлдэг контейнер модуль юм. - Энэ нь Python-ийн ердийн жагсаалтаас ялгаатай нь дэд модулиудыг зөв бүртгэж, параметрүүдийг харагдацтай болгодог.

In [1]:
import torch
import torch.nn as nn

class MyNetwork(nn.Module):
    def __init__(self):
        super(MyNetwork, self).__init__()
        
        # ModuleList ашиглан хэд хэдэн давхаргыг цэгцлэх
        self.layers = nn.ModuleList([
            nn.Linear(10, 20),
            nn.ReLU(),
            nn.Linear(20, 30),
            nn.ReLU(),
            nn.Linear(30, 5)
        ])
        
        # ModuleList-г хоосон эхлүүлж, дараа нь нэмэх
        self.extra_layers = nn.ModuleList()
        
    def forward(self, x):
        # ModuleList-ийн давхаргуудаар дамжуулан өгөгдөл дамжуулах
        for layer in self.layers:
            x = layer(x)
        return x

In [2]:
model = MyNetwork()
print(model)

MyNetwork(
  (layers): ModuleList(
    (0): Linear(in_features=10, out_features=20, bias=True)
    (1): ReLU()
    (2): Linear(in_features=20, out_features=30, bias=True)
    (3): ReLU()
    (4): Linear(in_features=30, out_features=5, bias=True)
  )
  (extra_layers): ModuleList()
)


## Гол онцлогууд:
### 1. Жагсаалтад модуль нэмэх

In [3]:
model = nn.ModuleList()

# Модулиуд нэмэх
model.append(nn.Linear(10, 20))
model.append(nn.ReLU())
model.append(nn.Linear(20, 5))

# Энгийн жагсаалтад нэмэх мэт ажиллана
print(f"Модулийн тоо: {len(model)}")

Модулийн тоо: 3


### 2. Индексээр модульд хандах

In [4]:
# Тухайн индекс дэх модулийг авах
first_layer = model[0]
last_layer = model[-1]

# Модулийг солих
model[1] = nn.LeakyReLU(0.1)

In [5]:
print(model)

ModuleList(
  (0): Linear(in_features=10, out_features=20, bias=True)
  (1): LeakyReLU(negative_slope=0.1)
  (2): Linear(in_features=20, out_features=5, bias=True)
)


### 3. Хэд хэдэн модуль нэмэх

In [6]:
# Олон модулийг нэг дор нэмэх
model.extend([
    nn.Dropout(0.5),
    nn.Linear(5, 2),
    nn.Softmax(dim=1)
])

ModuleList(
  (0): Linear(in_features=10, out_features=20, bias=True)
  (1): LeakyReLU(negative_slope=0.1)
  (2): Linear(in_features=20, out_features=5, bias=True)
  (3): Dropout(p=0.5, inplace=False)
  (4): Linear(in_features=5, out_features=2, bias=True)
  (5): Softmax(dim=1)
)

### 4. Модулийг оруулах

In [8]:
# Тодорхой байрлалд модуль оруулах
model.insert(2, nn.BatchNorm1d(20))
print(model)

ModuleList(
  (0): Linear(in_features=10, out_features=20, bias=True)
  (1): LeakyReLU(negative_slope=0.1)
  (2-3): 2 x BatchNorm1d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (4): Linear(in_features=20, out_features=5, bias=True)
  (5): Dropout(p=0.5, inplace=False)
  (6): Linear(in_features=5, out_features=2, bias=True)
  (7): Softmax(dim=1)
)


## Чухал зүйлс:
### ModuleList vs Python list

In [9]:
# Буруу арга (асуудал үүсгэдэг)
class WrongModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = []  # Энгийн жагсаалт ашиглах
        for i in range(3):
            self.layers.append(nn.Linear(10, 10))
        # Параметрүүд бүртгэгдэхгүй!

# Зөв арга
class CorrectModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = nn.ModuleList()  # ModuleList ашиглах
        for i in range(3):
            self.layers.append(nn.Linear(10, 10))
        # Параметрүүд автоматаар бүртгэгдэнэ!

### ModuleList-ийн параметрүүд

In [10]:
model = nn.ModuleList([
    nn.Linear(10, 20),
    nn.Linear(20, 30)
])

# Бүх параметрүүдийг хэвлэх
for param in model.parameters():
    print(param.shape)
# Гаралт: torch.Size([20, 10]), torch.Size([20]), torch.Size([30, 20]), torch.Size([30])

torch.Size([20, 10])
torch.Size([20])
torch.Size([30, 20])
torch.Size([30])


## Бодит хэрэглээний жишээ:
### Динамик сүлжээ бүтээх

In [11]:
class DynamicNetwork(nn.Module):
    def __init__(self, layer_sizes):
        super().__init__()
        self.layers = nn.ModuleList()
        
        # Оролт-гаралтын хэмжээсээс хамааран давхаргуудыг динамикаар үүсгэх
        for i in range(len(layer_sizes) - 1):
            self.layers.append(
                nn.Linear(layer_sizes[i], layer_sizes[i + 1])
            )
            # Сүүлийн давхаргад активацийн функцийг нэмэхгүй
            if i < len(layer_sizes) - 2:
                self.layers.append(nn.ReLU())
    
    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

# Хэрэглээ
sizes = [784, 256, 128, 64, 10]  # MNIST даалгаварт
model = DynamicNetwork(sizes)

### Residual холболттой сүлжээ

In [12]:
class ResidualBlock(nn.Module):
    def __init__(self, in_features, out_features):
        super().__init__()
        self.layers = nn.ModuleList([
            nn.Linear(in_features, out_features),
            nn.BatchNorm1d(out_features),
            nn.ReLU(),
            nn.Linear(out_features, out_features),
            nn.BatchNorm1d(out_features)
        ])
        
        # Shortcut connection
        self.shortcut = nn.Linear(in_features, out_features) if in_features != out_features else None
    
    def forward(self, x):
        residual = x
        for layer in self.layers:
            x = layer(x)
        
        if self.shortcut is not None:
            residual = self.shortcut(residual)
        
        return torch.relu(x + residual)

## Ашиг тус:

1. Параметрийн менежмент: Модулиудын параметрүүдийг автоматаар бүртгэдэг

2. Уян хатан байдал: Динамикаар давхарга нэмж/хасах боломжтой

3. Код цэвэрхэн: Олон тооны давхаргуудыг цэгцтэй хадгалах

4. Serialization бэлэн: Модулиудыг хадгалах/дуураахад бэлэн

## Анхааруулга:
- ModuleList нь forward() функцийг агуулаагүй - зөвхөн модулиудыг хадгалах зориулалттай

- Давхаргуудыг гогцоогоор дамжуулах шаардлагатай

- Индексээр хандахдаа модулийн төрлийг шалгах хэрэгтэй

Энэ нь ихэвчлэн динамик бүтэцтэй нейрон сүлжээ, давхаргуудын тоо оролтоос хамаарах моделуудад ашиглагддаг.